In [1]:
import pandas as pd
import numpy as np
import importlib
import get_started_doing_something_variables
import get_stopped_doing_something_variables
import prepare_for_the_models
import read_a_combination_of_variables
from dateutil.relativedelta import relativedelta
import get_all_vars
import yaml
import warnings
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
from helpers.s3_bucket_utils import S3BucketUtils
from helpers import settings

bucket = S3BucketUtils()

warnings.filterwarnings('ignore')
from IPython.display import display, Markdown

with open(r'./parameters/for_properly_used_inquiries_vars.yaml') as file:
    for_properly_used_inquiries_vars = yaml.load(file, Loader=yaml.FullLoader)
    
pd.set_option('display.max_colwidth', 5000)
def add_month(date, m):
    ddd = pd.to_datetime(date, format='%Y-%m-%d')
    ddd2 = ddd + relativedelta(months=m)
    return (str(ddd2))[0:10]

def get_key_based_on_value_in_a_dict(dict_, value_of_interest):
    for key, value in dict_.items():
        if value == value_of_interest:
            return key

def get_model_number(var_, model_names):
    if len(model_names[model_names['model_name']==var_]['model_number'])>0:
        model_number = model_names[model_names['model_name']==var_]['model_number'].iloc[0]
    else:
        multiple_vars = model_names[(model_names['model_name'].apply(lambda x: ',' in x))]
        for i in range(0, len(multiple_vars)):
            if var_ in multiple_vars['model_name'].iloc[i]:
                model_number = multiple_vars['model_number'].iloc[i]
                break
    return model_number

def get_behavioural_vars(churn_based_on_behaviour_dir, date_dir):
    vars_periods_to_look_at_thresholds_to_use = \
    bucket.load_csv_from_s3(file_name = churn_based_on_behaviour_dir + 'data/' + date_dir +\
                     '/vars_periods_to_look_at_thresholds_to_use.csv')

    period_to_look_at_stopped = \
    vars_periods_to_look_at_thresholds_to_use[['variable_base_name', 'period_to_look_at_stopped', 'number_of_months_stopped']].\
    apply(lambda x: (x['variable_base_name'], x['period_to_look_at_stopped']), axis=1).tolist()
    
    period_to_look_at_started = \
    vars_periods_to_look_at_thresholds_to_use[['variable_base_name', 'period_to_look_at_started', 'number_of_months_started']].\
    apply(lambda x: (x['variable_base_name'], x['period_to_look_at_started']), axis=1).tolist()
    

    (all_vars, all_vars_base_names) = get_all_vars.main()
    before_var = dict.fromkeys(all_vars_base_names.values())
    last_X_months_var = dict.fromkeys(all_vars_base_names.values())
    before_and_didnt_last_X_months = dict.fromkeys(all_vars_base_names.values())

    for i in range(0, len(period_to_look_at_stopped)):
        before_var[period_to_look_at_started[i][0]] = 'had_'+period_to_look_at_started[i][0]+'_before'
        last_X_months_var[period_to_look_at_stopped[i][0]] = 'had_'+period_to_look_at_stopped[i][0]+'_'+period_to_look_at_stopped[i][1]
        before_and_didnt_last_X_months[period_to_look_at_stopped[i][0]] = \
        'had_'+period_to_look_at_stopped[i][0]+'_before_and_didnt_'+period_to_look_at_stopped[i][1]

    for key in before_var.keys():
        if 'changed' in key.lower() and 'inquiry' in key.lower():
            period_to_look_at_stopped_tmp = \
            vars_periods_to_look_at_thresholds_to_use[vars_periods_to_look_at_thresholds_to_use['variable_base_name']==key]['period_to_look_at_stopped'].iloc[0]
            
            period_to_look_at_started_tmp = \
            vars_periods_to_look_at_thresholds_to_use[vars_periods_to_look_at_thresholds_to_use['variable_base_name']==key]['period_to_look_at_started'].iloc[0]

            before_var[key] = 'had_'+for_properly_used_inquiries_vars['changed_inquiry_status_to_properly_used'][key]+'_before'
            last_X_months_var[key] = \
            'had_'+for_properly_used_inquiries_vars['changed_inquiry_status_to_properly_used'][key]+'_'+period_to_look_at_stopped_tmp
            before_and_didnt_last_X_months[key] = \
            'had_'+for_properly_used_inquiries_vars['changed_inquiry_status_to_properly_used'][key]+'_before_and_didnt_'+period_to_look_at_stopped_tmp

            
    return (before_var, last_X_months_var, before_and_didnt_last_X_months)

def get_vars_for_the_report(df): ##returns vars with 1/0 values
    tv_vars = []
    fixed_vars = []
    for col in df:
        if df[col].nunique()==2 and 1 in df[col].unique() and 0 in df[col].unique() and col not in ['event', 'premium_service_hs', 'requested_cancellation']:
            if (len(df.drop_duplicates(['spot_id', col])) == df['spot_id'].nunique()):
                fixed_vars.append(col)
            else:
                tv_vars.append(col)
            
    return (tv_vars, fixed_vars)

def get_report_with_number_of_spots_with_var_set_to_yes(vars_of_interest, tv_vars, fixed_vars, df, df_with_last_month, last_month):
    number_of_spots_with_var_yes = pd.DataFrame(columns = ['variable', 'spots_for_the_model', \
                                                       'premium_spots',\
                                                       'premium_spots_which_requested_cancellation',\
                                                       'spots_which_requested_cancellation',\
                                                       'spots_which_cancelled',\
                                                           'spots_with_variable_set_to_yes_month_before_the_event',\
                                                        'spots_with_variable_set_to_yes_last_month',\
                                                           'last_month_%_of_all_premium_spots',\
                                                           'last_month_%_of_all_spots',\
                                                      # 'number_of_spots_with_variable_set_to_yes_for_fixed_vars',\
                                                      #'number_of_spots_with_variable_set_to_yes_for_fixed_vars_%',\
                                                          ])
                                                      
    number_of_spots_with_var_yes['variable'] = vars_of_interest
    number_of_spots_with_var_yes.set_index('variable', inplace = True)

    fixed_vars = [x for x in vars_of_interest if x in fixed_vars]
    tv_vars = [x for x in vars_of_interest if x in tv_vars]
    
    for var in vars_of_interest:
        if var in fixed_vars:
            number_of_spots_with_var_yes.loc[var, 'spots_with_variable_set_to_yes_for_fixed_vars'] = \
            df[df[var]==1]['spot_id'].nunique()

        number_of_spots_with_var_yes.loc[var, 'spots_with_variable_set_to_yes_last_month'] = \
        df_with_last_month[(df_with_last_month['left_limit']==last_month)&\
                   (df_with_last_month['spot_id'].isin(df['spot_id'].unique()))&\
                   (df_with_last_month[var]==1)]['spot_id'].nunique()
        
        number_of_spots_with_var_yes.loc[var, 'spots_with_variable_set_to_yes_month_before_the_event'] = \
        df[(df[var]==1)&\
          (df['event']==True)]['spot_id'].nunique()

        for_weighted_avg = df[(df[var]==1)][['left_limit', 'spot_id']].\
        groupby('left_limit')['spot_id'].nunique().reset_index().rename(columns = {'spot_id':'number_of_spots', 'left_limit':'month'})

        for_weighted_avg['total_number_of_spots_with_var_set_to_yes'] = df[df[var]==1]['spot_id'].nunique()
        for_weighted_avg['total_number_of_spots'] = df['spot_id'].nunique()
        #for_weighted_avg['variable'] = var
        for_weighted_avg = for_weighted_avg.sort_values('number_of_spots', ascending = True).reset_index(drop = True)
        for_weighted_avg['weights'] = for_weighted_avg.index + 1
        for_weighted_avg['weights_sum'] = for_weighted_avg['weights'].sum()
        for_weighted_avg['number_of_spots_%'] = (for_weighted_avg['number_of_spots']/for_weighted_avg['total_number_of_spots'])*100
        for_weighted_avg['weights'] = for_weighted_avg['weights']/for_weighted_avg['weights_sum']


        regular_mean = for_weighted_avg['number_of_spots'].mean()
        regular_mean_perc = for_weighted_avg['number_of_spots_%'].mean()

        weighted_mean = (for_weighted_avg['number_of_spots']*for_weighted_avg['weights']).sum()
        weighted_mean_perc = (for_weighted_avg['number_of_spots_%']*for_weighted_avg['weights']).sum()

        number_of_spots_with_var_yes.loc[var, 'spots_with_variable_set_to_yes_regular_mean'] = round(regular_mean)
        number_of_spots_with_var_yes.loc[var, '%_of_total_spots_regular_mean'] = round(regular_mean_perc, 2)
        number_of_spots_with_var_yes.loc[var, 'spots_with_variable_set_to_yes_weighted_mean'] = round(weighted_mean)
        number_of_spots_with_var_yes.loc[var, '%_of_total_spots_weighted_mean'] = round(weighted_mean_perc, 2)

    number_of_spots_with_var_yes['spots_for_the_model'] = df_timeline_with_add_fields['spot_id'].nunique()
    number_of_spots_with_var_yes['premium_spots'] = df_timeline_with_add_fields[df_timeline_with_add_fields['premium_service_hs']==True]['spot_id'].nunique()
    number_of_spots_with_var_yes['premium_spots_which_requested_cancellation'] = \
    df_timeline_with_add_fields[(df_timeline_with_add_fields['premium_service_hs']==True)&\
                               (df_timeline_with_add_fields['requested_cancellation']==True)]['spot_id'].nunique()
    number_of_spots_with_var_yes['spots_which_requested_cancellation'] = \
    df_timeline_with_add_fields[df_timeline_with_add_fields['requested_cancellation']==True]['spot_id'].nunique()
    number_of_spots_with_var_yes['spots_which_cancelled'] = \
    df_timeline_with_add_fields[df_timeline_with_add_fields['premium_service_hs']==False]['spot_id'].nunique()
    

    number_of_spots_with_var_yes['last_month_%_of_all_spots'] = \
    round((number_of_spots_with_var_yes['spots_with_variable_set_to_yes_last_month']/number_of_spots_with_var_yes['spots_for_the_model'])*100, 2)

    number_of_spots_with_var_yes['last_month_%_of_all_premium_spots'] = \
    round((number_of_spots_with_var_yes['spots_with_variable_set_to_yes_last_month']/number_of_spots_with_var_yes['premium_spots'])*100, 2)
    
    return number_of_spots_with_var_yes


# read the yaml file with a list of parameters needed for the report
with open(r'./parameters/started_doing_something_report_parameters.yaml') as file:
    parameters = yaml.load(file, Loader=yaml.FullLoader)

date_of_analysis = parameters['date_of_analysis']
date_dir = date_of_analysis.replace('-', '_')
### name of the data directory ###
churn_based_on_behaviour_dir = parameters['churn_based_on_behaviour_dir']

In [2]:
##### base columns ##### 
base_cols = ['spot_id',\
            'time',\
            'event']


data_sets = [#'ALL_spots_with_CB_cancellation_confirmed',\
             'ALL_spots_with_CB_cancellation_requested',\
             #'ALL_spots_wo_CB_cancellation_confirmed', \
             #'ALL_spots_wo_CB_cancellation_requested',\
            # 'CAN_CANCEL_spots_with_CB_cancellation_confirmed',\
            # 'CAN_CANCEL_spots_with_CB_cancellation_requested',\
             #'CAN_CANCEL_spots_wo_CB_cancellation_confirmed', \
             'CAN_CANCEL_spots_wo_CB_cancellation_requested']

for data_set_name in data_sets:

    with open(r'./parameters/data_sets.yaml') as file:
        data_sets_parameters = yaml.load(file, Loader=yaml.FullLoader)

    spots_set = data_sets_parameters[data_set_name]['spots_set']
    with_wo_CB = data_sets_parameters[data_set_name]['with_wo_CB']
    event_date_full_name = data_sets_parameters[data_set_name]['event_date_type']

    if with_wo_CB == 'with_CB':
        with_wo_CB_boolean = True
    else:
        with_wo_CB_boolean = False

    if event_date_full_name == 'cancellation_requested':
        event_date = 'canc_req'
    elif event_date_full_name == 'cancellation_confirmed':
        event_date = 'canc_conf'

    ##### Model 0: all variables - prepare data for the model ####
    cols_to_use = read_a_combination_of_variables.\
    main(model_number=0, dir_name='combinations_of_variables_that_are_not_dependent/')
    #### get behavioural variables ####
    (variables_to_use_for_the_model, did_something_last_month_vars, did_something_before_and_didnt_last_month_vars) = \
    get_started_doing_something_variables.main(date_of_analysis=date_of_analysis, variables_to_use_for_the_model=cols_to_use)

    (variables_to_use_for_the_model, did_something_before_vars, did_something_last_month_vars) = \
    get_stopped_doing_something_variables.main(date_of_analysis=date_of_analysis, variables_to_use_for_the_model=cols_to_use)


    cols = base_cols + variables_to_use_for_the_model + did_something_before_vars + \
    did_something_before_and_didnt_last_month_vars + did_something_last_month_vars

    #### get data for the model ###
    (data, base_df, df_timeline_all_vars) = \
    prepare_for_the_models.get_data_for_the_MV_Cox_model(date_of_analysis=date_of_analysis, spots_set=spots_set, \
                                with_wo_CB=with_wo_CB, event_date=event_date, columns=cols, data_dir=churn_based_on_behaviour_dir,\
                                C = 100)

    additional_fields = ['limits', 'left_limit', 'right_limit', 'premium_service_hs', 'requested_cancellation', 'end', 'cancellation_requested']
    df_timeline_with_add_fields = df_timeline_all_vars.merge(data[['spot_id', 'time']+additional_fields].rename(columns = {'time':'stop'}),\
                                   on = ['spot_id', 'stop'])
    df_timeline_with_add_fields['their_own_website_yes'] = 0
    df_timeline_with_add_fields.loc[(df_timeline_with_add_fields['their_own_website_dontKnow']==0)&\
                                   (df_timeline_with_add_fields['their_own_website_no']==0), 'their_own_website_yes'] = 1

    (tv_vars, fixed_vars) = get_vars_for_the_report(df_timeline_with_add_fields)
    (before_var, last_X_months_var, before_and_didnt_last_X_months) = \
    get_behavioural_vars(churn_based_on_behaviour_dir=churn_based_on_behaviour_dir,\
                           date_dir=date_dir)

    with open(r'./parameters/base_vars_for_the_num_of_vars_set_to_yes_export.yaml') as file:
        base_names_of_vars_of_interest = yaml.load(file, Loader=yaml.FullLoader)
        base_names_of_vars_of_interest = \
        base_names_of_vars_of_interest['base_names_of_vars_of_interest']

    vars_of_interest = []
    for base_var in base_names_of_vars_of_interest:
        vars_of_interest.append(before_var[base_var])
        vars_of_interest.append(last_X_months_var[base_var])
        vars_of_interest.append(before_and_didnt_last_X_months[base_var])

    number_of_spots_with_var_yes = \
    get_report_with_number_of_spots_with_var_set_to_yes(vars_of_interest=vars_of_interest,\
                                                        tv_vars=tv_vars, fixed_vars=fixed_vars, \
                                                        df=df_timeline_with_add_fields,\
                                                        df_with_last_month=data, last_month=add_month(date_of_analysis, -1))

    number_of_spots_with_var_yes.\
    to_csv('data/'+date_dir+'/exports/number_of_spots_with_var_set_to_yes_'+data_set_name+'.csv')